# Tutorial: Aperiodic Data and Jackknifes

We present a basic example of the use of the RascalC Python wrapper for a single field, computing the 2PCF in $(r,\mu)$ bins, and calibrating for the non-Gaussianity parameter using jackknifes.

Here, we compute the covariance matrix for a single [QPM mock](https://arxiv.org/pdf/1309.5532.pdf) dataset.

## Installing dependencies

You need to install the `RascalC` Python library to run this notebook.
It can be done via

In [ ]:
! python3 -m pip install https://github.com/misharash/RascalC.git

The code requires [`pycorr`](https://github.com/cosmodesi/pycorr) to deal with pair counts and data correlation function estimators.
To compute pair counts of catalogs (which is part of this tutorial), you need a [custom version of `Corrfunc`](https://github.com/adematti/Corrfunc) (see also [`pycorr` installation instructions](https://py2pcf.readthedocs.io/en/latest/user/building.html)).
Both can be installed quickly via

In [ ]:
! python3 -m pip install 'git+https://github.com/cosmodesi/pycorr#egg=pycorr[corrfunc]'

More common requirements are `astropy` and `scikit-learn`.
They can be installed through Anaconda (`conda`) if you have it:

In [ ]:
! conda install astropy scikit-learn

Otherwise, install via `pip`:

In [ ]:
! python3 -m pip install astropy scikit-learn

## Getting the input data

The following cells download the publicly available mock galaxy and random catalogs for [BOSS DR12](https://www.sdss3.org/science/boss_publications.php) from the [SDSS website](https://sdss.org/).
If you already have `mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw` and `mock_random_DR12_CMASS_N_50x1.rdzw` files, please go ahead to the next section.

Beware that the archive with the mock data is about 12 gigabytes, although it can be removed after extracting just one catalog of about 37 megabytes.
The random catalog is about 1.5 gigabytes uncompressed.

In [1]:
! wget https://data.sdss.org/sas/dr12/boss/lss/qpm_mocks/mock_galaxy_DR12_CMASS_N_QPM_allmocks.tar.gz # download the archive with all mock galaxy catalogs (≈12 GB)
! tar -xzvf mock_galaxy_DR12_CMASS_N_QPM_allmocks.tar.gz mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw # unpack one catalog (≈37 MB)
! rm mock_galaxy_DR12_CMASS_N_QPM_allmocks.tar.gz # remove the big (≈12 GB) archive

--2024-04-30 11:55:58--  https://data.sdss.org/sas/dr12/boss/lss/qpm_mocks/mock_galaxy_DR12_CMASS_N_QPM_allmocks.tar.gz
Resolving data.sdss.org (data.sdss.org)... 155.101.19.31
Connecting to data.sdss.org (data.sdss.org)|155.101.19.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12482301869 (12G) [application/octet-stream]
Saving to: ‘mock_galaxy_DR12_CMASS_N_QPM_allmocks.tar.gz’

mock_galaxy_DR12_CM 100%[===================>]  11.62G  75.9MB/s    in 3m 11s  

2024-04-30 11:59:10 (62.2 MB/s) - ‘mock_galaxy_DR12_CMASS_N_QPM_allmocks.tar.gz’ saved [12482301869/12482301869]

^C


In [2]:
! wget https://data.sdss.org/sas/dr12/boss/lss/qpm_mocks/mock_random_DR12_CMASS_N_50x1.rdzw.gz # download the compressed random catalog (≈600 MB)
! gunzip mock_random_DR12_CMASS_N_50x1.rdzw.gz # uncompress (≈1.5 GB), typically deleting the compressed version

--2024-04-30 12:01:32--  https://data.sdss.org/sas/dr12/boss/lss/qpm_mocks/mock_random_DR12_CMASS_N_50x1.rdzw.gz
Resolving data.sdss.org (data.sdss.org)... 155.101.19.31
Connecting to data.sdss.org (data.sdss.org)|155.101.19.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600910232 (573M) [application/octet-stream]
Saving to: ‘mock_random_DR12_CMASS_N_50x1.rdzw.gz’

mock_random_DR12_CM 100%[===================>] 573.07M  54.2MB/s    in 9.7s    

2024-04-30 12:01:41 (59.2 MB/s) - ‘mock_random_DR12_CMASS_N_50x1.rdzw.gz’ saved [600910232/600910232]



## Preparing the catalogs

Set the filenames

In [1]:
galaxies_filename = "mock_galaxy_DR12_CMASS_N_QPM_0001.rdzw"
randoms_filename = "mock_random_DR12_CMASS_N_50x1.rdzw"

Read the original files: text with RA, DEC, Z (redshift) and weight columns.
This, and many of the next cells, may take about 10 seconds because of the large number of randoms.

In [2]:
import numpy as np
from astropy.table import Table
galaxies = Table(np.loadtxt(galaxies_filename, usecols = range(4)), names = ["RA", "DEC", "Z", "WEIGHT"]) # ignore the last column, not sure what it is
randoms = Table(np.loadtxt(randoms_filename, usecols = range(4)), names = ["RA", "DEC", "Z", "WEIGHT"])

Compute the comoving distance within the fiducial (grid) cosmology. Here we use a utility function from the RascalC library to do this.

In [3]:
from RascalC.pre_process.convert_to_xyz import comoving_distance_Mpch
Omega_m = 0.29; Omega_k = 0; w_DE = -1 # density parameters of matter and curvature, and the equation-of-state parameter of dark energy
galaxies["comov_dist"] = comoving_distance_Mpch(galaxies["Z"], Omega_m, Omega_k, w_DE)
randoms["comov_dist"] = comoving_distance_Mpch(randoms["Z"], Omega_m, Omega_k, w_DE)

Let us define a utility function for position formatting that will be useful on several more occasions.

In [4]:
def get_rdd_positions(catalog: Table) -> tuple[np.ndarray[float]]: # utility function to format positions from a catalog
    return (catalog["RA"], catalog["DEC"], catalog["comov_dist"])

Assign jackknife regions to both galaxies and randoms.

The `get_subsampler_xirunpc` function generates a $K$-means subsampler from `sklearn` under the rug through a `pycorr` interface, ensuring that it is run single-threaded.
If you call it in your own way, beware that it might run multi-threaded if `OMP_` environment variables are set, and that is known to impede the performance of the main `RascalC` covariance computation later (due to OpenMP limitations).

$K$-means is nice in that it can generate a fixed number of regions of similar size with realistic survey geometry.
(However, it is not without issues, e.g. it does not guarantee similar completeness patterns that may affect the shot-noise rescaling.)
Previously, jackknife region numbers were assigned as `healpix` pixels (with number controlled by `NSIDE` variable), which is simpler, but less balanced and flexible.
In cubic/rectangular boxes, box subsamplers from `pycorr` may be useful.
If you have better ideas, feel free to use them and perhaps share with the code authors if they work out well.

In [5]:
from RascalC.pre_process.create_jackknives_pycorr import get_subsampler_xirunpc
n_jack = 60 # number of regions
subsampler = get_subsampler_xirunpc(get_rdd_positions(galaxies), n_jack, position_type = "rdd") # "rdd" means RA, DEC in degrees and then distance (corresponding to pycorr)
galaxies["JACK"] = subsampler.label(get_rdd_positions(galaxies), position_type = "rdd")
randoms["JACK"] = subsampler.label(get_rdd_positions(randoms), position_type = "rdd")

Select a smaller subset of randoms to make pair counting and `RascalC` importance sampling more feasible

In [6]:
x_randoms = 10 # how many times the number of galaxies should the number of randoms be; the total number of randoms is ≈50x the number of galaxies
np.random.seed(42) # for reproducibility
randoms_subset = randoms[np.random.choice(len(randoms), x_randoms * len(galaxies), replace = False, p = randoms["WEIGHT"] / np.sum(randoms["WEIGHT"]))]

## Pair counts and correlation functions with [`pycorr`](https://github.com/cosmodesi/pycorr)

In addition to the galaxy and randoms catalogs, `RascalC` requires the random counts and correlation functions.
We use [`pycorr`](https://github.com/cosmodesi/pycorr) — a wrapper over the fast pair-counting engine `Corrfunc` that saves all the counts and allows to re-bin.
So it can nicely do in one go what used to require several scripts in the legacy version of this tutorial.

We remind you to make sure your current Python environment has [the custom version of `Corrfunc`](https://github.com/cosmodesi/Corrfunc) as suggested for [`pycorr`](https://github.com/cosmodesi/pycorr) (see also [the Installing dependencies section](#Installing-dependencies) in the beginning of this notebook).

Pair counting requires the same catalogs as the main `RascalC` covariance computation.
To minimize the repetitions and/or avoid splitting in this tutorial, we decided to incorporate the `pycorr` run here.
This required some extra tricks to avoid the OpenMP (multi-threading) interference, even more so to make this run safely in a Jupyter notebook (and not just in a script).

In a "production" run, you might want to set up the `pycorr` pair counting separately.
We typically do so, noting that it can be performed on GPU while the main covariance computation with `RascalC` proper requires CPU only.
But it is **very important** to reproduce the pre-processing (including jackknife assignment) for `RascalC` inputs in the same way it was done for pair counting.

First, we choose whether to use full randoms or a smaller subset for pair counting.
The latter is faster but a bit less precise.

In [7]:
# randoms_for_counts = randoms
randoms_for_counts = randoms_subset

We continue by splitting the randoms into parts of roughly the same size as data.
This gives high precision at fixed computing cost [(Keihänen et al 2019)](https://arxiv.org/abs/1905.01133).

In [8]:
n_splits = int(np.rint(len(randoms_for_counts) / len(galaxies))) # the number of parts to split the randoms to
print(f"Splitting randoms into {n_splits} parts")

# split randoms into the desired number of parts randomly
random_indices = np.arange(len(randoms_for_counts))
np.random.seed(42) # for reproducibility
np.random.shuffle(random_indices) # random shuffle in place
random_parts = [randoms_for_counts[random_indices[i_random::n_splits]] for i_random in range(n_splits)] # quick way to produce parts of almost the same size

# normalize the weights in each part — fluctuations in their sums may be a bit of a problem
for i_random in range(n_splits): random_parts[i_random]["WEIGHT"] /= np.sum(random_parts[i_random]["WEIGHT"])

Splitting randoms into 10 parts


Import libraries and select settings

In [9]:
from pycorr import TwoPointCorrelationFunction, setup_logging
from tqdm import trange # nice progress bar
import os

In [9]:
n_threads = 10 # number of threads for pycorr computation
s_max = 200 # maximal separation in Mpc/h
n_mu = 200 # number of angular (µ) bins
counts_filename = f"allcounts_mock_galaxy_DR12_CMASS_N_QPM_0001_lin_njack{n_jack}_nran{n_splits}_split{0}.npy" # filename to save counts

Define the function to compute the counts, using split randoms at larger separations.

In [10]:
s_edges = np.arange(s_max + 1) # 1 Mpc/h wide separation bins from 0 to s_max Mpc/h
mu_edges = np.linspace(-1, 1, n_mu + 1) # make uniform µ bins between -1 and 1, or twice less bins between 0 and 1 after wrapping (will be done within RascalC wrapper)

def run_pair_counts(): # the code to run pycorr, needs to be invoked in a separate process from what will run RascalC then!
    setup_logging()
    result = 0
    D1D2 = None # to compute the data-data counts on the first go but not recompute then
    for i_random in trange(n_splits, desc="Computing counts with random part"):
        these_randoms = random_parts[i_random]
        tmp = TwoPointCorrelationFunction(mode = 'smu', edges = (s_edges, mu_edges),
                                        data_positions1 = get_rdd_positions(galaxies), data_weights1 = galaxies["WEIGHT"], data_samples1 = galaxies["JACK"],
                                        randoms_positions1 = get_rdd_positions(these_randoms), randoms_weights1 = these_randoms["WEIGHT"], randoms_samples1 = these_randoms["JACK"],
                                        position_type = "rdd", engine = "corrfunc", D1D2 = D1D2, gpu = False, nthreads = n_threads)
        # "rdd" means RA, DEC in degrees and then distance
        D1D2 = tmp.D1D2 # once computed, becomes not None and will not be recomputed
        result += tmp
    result.D1D2.attrs['nsplits'] = n_splits

    result.save(counts_filename)

Next is a tricky part where we create a child process to run pair counts to avoid OpenMP multi-threading interference.

The computation does take a while (about 10 minutes for me at NERSC login node; expect to see 1/10 progress in a few minutes) even with multi-threading (which I haven't managed to make work with `Corrfunc` in macOS yet).
If you already have the counts saved into file, no need to run this cell again – they will be loaded later.

It is **best not to interrupt the next cell** because it is rather hard to control the forked child process.

In [11]:
child_pid = os.fork() # creates a new process at the same execution stage with access to all the data. Returns 0 in child process and non-zero child process ID in the original, parent process
if not child_pid:
    # new child process
    print(f"Child process ID {os.getpid()}") # printing from the parent process may interfere with the progress bar
    import traceback
    try:
        run_pair_counts()
    except Exception:
        traceback.print_exc()
        os._exit(-1) # terminate the process with error
    # in a script, would not need try-except and import traceback above. This is because script process is already terminated on error/exception with a non-zero exit code, in notebook it remains to hang around
    os._exit(0) # terminate ok if no error occured
else:
    # original parent process
    child_pid2, child_status = os.waitpid(child_pid, 0) # wait for the child process to terminate
    if child_status: raise RuntimeError(f"Child process exited with error (code {os.waitstatus_to_exitcode(child_status)}). See its traceback above.")

Child process ID 93491


Computing xi with random part: 100%|██████████| 10/10 [09:07<00:00, 54.73s/it]


**In case the previous cell was interrupted**, the child process may remain hanging about, so **execute the following cell** to force it to terminate.
(Unlikely to harm to execute it otherwise.)

In [ ]:
import signal
os.kill(child_pid, signal.SIGKILL)

Finally, we should load the saved counts in the original process. You can continue from this step if you computed the counts before.

In [12]:
allcounts = TwoPointCorrelationFunction.load(counts_filename)

## Covariance settings and computation

Mode parameters

In [13]:
mode = "legendre_projected" # Legendre multipoles projected from s,µ bins
max_l = 4 # max multipole to compute the covariance for
periodic_boxsize = None # not a periodic box

Runtime and convergence parameters

In [20]:
n_threads = 10
N2 = 10 # number of secondary points sampled per primary random point
N3 = 20 # number of tertiary points sampled per each secondary
N4 = 40 # number of quaternary points sampled per each tertiary
n_loops = 80 # must be divisible by n_threads
loops_per_sample = 5 # must divide n_loops

We remind that `RascalC` will be provided with only a subset of randoms here, `randoms_subset`.

The convergence of covariance matrix integrals is determined by the number of 4-point configurations sampled, which is roughly `len(random_subset)*N2*N3*N4*n_loops`.

The runtime is roughly proportional to the same number of 4-point configurations but divided by the number of threads: `len(random_subset)*N2*N3*N4*n_loops/n_thread`.
* Different loops may take different time to complete due to random factors, so it is recommended to have the number of loops several times the number of threads.
* It is not advisable to set either of `N2/3/4` too low, say `<5`, because then loops may become inefficient.

Finally, `loops_per_sample` regulates how many loops are grouped into a distinct estimate of covariance matrices using a subset of sampled point configurations.
The code then needs some (say 10-30) such independent subsamples to estimate the convergence (and inversion bias) of the covariance matrix, but saving too many increases disk usage and can slow down the calculation.
A sample that is already saved can be reused (although this may be a bit tedious), otherwise there is no capability to restore the progress of `RascalC` after interruption/termination.

In the next cell we set the radial/separation binning of the covariance matrix by rebinning the counts. Here we leave the original number of angular bins – the covariance will be project into only a few (even) multipoles using all of them.

In [15]:
ds_cov = 4
s_min_cov = 20
s_max_cov = 200
allcounts_rebinned_cov = allcounts[s_min_cov:s_max_cov:ds_cov]

Separation bin edges for the covariance will be:

In [36]:
allcounts_rebinned_cov.edges[0]

array([ 20.,  24.,  28.,  32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,
        64.,  68.,  72.,  76.,  80.,  84.,  88.,  92.,  96., 100., 104.,
       108., 112., 116., 120., 124., 128., 132., 136., 140., 144., 148.,
       152., 156., 160., 164., 168., 172., 176., 180., 184., 188., 192.,
       196., 200.])

Also rebin the pycorr counts into reasonable (typically a bit finer) bins for the input two-point correlation function.
Here we also should reduce the number of angular ($\mu$) bins, because `RascalC` takes the input 2PCF as a $\xi(s,\mu)$ table in any mode.

It is also possible to generate a $\xi(s,\mu)$ table from theory and not from pycorr measurement.

In [16]:
ds_xi = 2
n_mu_xi = 20 # between µ = 0 and 1, i.e. after wrapping
assert allcounts.wrap().shape[1] % n_mu_xi == 0, "Counts not rebinnable to the desired number of angular bins"
allcounts_rebinned_xi = allcounts[::ds_xi, ::allcounts.wrap().shape[1] // n_mu_xi]

In [38]:
allcounts_rebinned_xi.edges

[array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
         22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
         44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
         66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
         88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
        110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
        132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
        154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
        176., 178., 180., 182., 184., 186., 188., 190., 192., 194., 196.,
        198., 200.]),
 array([-1.  , -0.95, -0.9 , -0.85, -0.8 , -0.75, -0.7 , -0.65, -0.6 ,
        -0.55, -0.5 , -0.45, -0.4 , -0.35, -0.3 , -0.25, -0.2 , -0.15,
        -0.1 , -0.05,  0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,
         0.35,  0.4 ,  0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,
         0.8 ,  0.85,  0.9 ,

Output and temporary directories

In [17]:
outdir = "out"
tmpdir = "tmp"

### The main covariance computation with `RascalC` interface

You should see several `Computing correlation function iteration` messages in a few minutes.
The whole computation will take a while (about 5 hours for me).

In [21]:
from RascalC import run_cov
results = run_cov(mode = mode, max_l = max_l, boxsize = periodic_boxsize,
                  nthread = n_threads, N2 = N2, N3 = N3, N4 = N4, n_loops = n_loops, loops_per_sample = loops_per_sample,
                  pycorr_allcounts_11 = allcounts_rebinned_cov,
                  xi_table_11 = allcounts_rebinned_xi,
                  no_data_galaxies1 = len(galaxies),
                  position_type = "rdd",
                  randoms_positions1 = get_rdd_positions(randoms_subset), randoms_weights1 = randoms_subset["WEIGHT"], randoms_samples1 = randoms_subset["JACK"],
                  normalize_wcounts = True,
                  out_dir = outdir, tmp_dir = tmpdir)

Mode: legendre_projected
Periodic box: False
Jackknife: True
Number of tracers: 1
Normalizing weights and weighted counts: True
2024-05-01 07:29:32.927631
Wrapping pycorr_allcounts_11 to µ>=0
Number(s) of data galaxies: (642051,)
Wrapping xi_table_11 to µ>=0
2024-05-01 07:30:03.022447
Launching the C++ code with command: env OMP_PROC_BIND=spread OMP_PLACES=threads /global/common/software/desi/users/mrash/RascalC/RascalC/bin/cov.legendre_projected_jackknife -output out/ -nside 301 -rescale 1 -nthread 10 -maxloops 80 -loopspersample 5 -N2 10 -N3 20 -N4 40 -xicut 250 -binfile out/radial_binning_cov.csv -binfile_cf out/radial_binning_corr.csv -mbin_cf 20 -cf_loops 10 -in tmp/0.txt -norm 642051 -cor out/xi/xi_11.dat -max_l 4 -mu_bin_legendre_file out/weights/mu_bin_legendre_factors_m100_l4.txt -RRbin out/weights/binned_pair_counts_n45_m100_j60_11.dat -mbin 100 -jackknife out/weights/jackknife_weights_n45_m100_j60_11.dat

Mu bin Legendre factors file 'out/weights/mu_bin_legendre_factors_m100

ValueError: The full covariance is not positive definite - insufficient convergence

If the above ran well, most of the job is basically done, there are some additional explanations in the end.

It is quite possible that the convergence was insufficient above or the process timed out, instructions for that case follow.

The temporary directory can (and should) be removed safely in any case:

In [22]:
os.system(f"rm -rf {tmpdir}")

0

### In case of interruption or error

It may be useful to run the post-processing separately:

In [34]:
from RascalC.post_process.legendre_mix_jackknife import post_process_legendre_mix_jackknife
n_mu_orig = allcounts_rebinned_cov.wrap().shape[1]
n_s_bins = allcounts_rebinned_cov.shape[0]
post_process_legendre_mix_jackknife(f"{outdir}/xi_jack/xi_jack_n{n_s_bins}_m{n_mu_orig}_j{n_jack}_11.dat", f"{outdir}/weights", outdir, n_mu_orig, max_l, outdir)

Loading correlation function jackknife estimates from out/xi_jack/xi_jack_n45_m100_j60_11.dat
Loading jackknife weights from out/weights/jackknife_weights_n45_m100_j60_11.dat
Computing data covariance matrix
Loading mu bin Legendre factors from out/weights/mu_bin_legendre_factors_m100_l4.txt
Loading best estimate of jackknife covariance matrix
Optimizing for the shot-noise rescaling parameter


/global/common/software/desi/users/mrash/RascalC/RascalC/post_process/utils.py:46: UserWarning: Jackknife 4-point covariance matrix has not converged properly via the eigenvalue test. Min eigenvalue of C4 = -1.76e-05, min eigenvalue of C2 = 1.29e-08
  warn(f"{kind}4-point covariance matrix has not converged properly via the eigenvalue test. Min eigenvalue of C4 = {min(eig_c4):.2e}, min eigenvalue of C2 = {min(eig_c2):.2e}")


Optimization terminated successfully.
         Current function value: -70779206658080.171875
         Iterations: 66
         Function evaluations: 138
Optimization complete - optimal rescaling parameter is 1.067648


/global/common/software/desi/users/mrash/RascalC/RascalC/post_process/utils.py:46: UserWarning: Full 4-point covariance matrix has not converged properly via the eigenvalue test. Min eigenvalue of C4 = -2.33e-05, min eigenvalue of C2 = 1.73e-08
  warn(f"{kind}4-point covariance matrix has not converged properly via the eigenvalue test. Min eigenvalue of C4 = {min(eig_c4):.2e}, min eigenvalue of C2 = {min(eig_c2):.2e}")


ValueError: The full covariance is not positive definite - insufficient convergence

What to do with insufficient convergence

* run longer (list parameters)
* reduce the number of bins:
  * discard lower separation bins (names of parameters for that)
  * discard higher multipole moments (names of parameters for that too)
  * change the bin configuration more generally and re-launch the main covariance computation

### When you have converged and post-processed results

Everything returned by post-processing script is already saved in a compressed Numpy file in the output directory and can be loaded again as

In [ ]:
results = np.load(f"{outdir}/Rescaled_Covariance_Matrices_Legendre_Jackknife_n{n_s_bins}_l{max_l}_j{n_jack}.npz")

Optimal shot-noise rescaling value can be accessed as

In [ ]:
results["shot_noise_rescaling"]

Final covariance matrix is stored as

In [ ]:
results["full_theory_covariance"]

For historical reasons, its ordering is (radial bin, multipole) with only even multipoles included.

We provide a utility function for transposing it to (multipole, radial bin) ordering:

In [ ]:
from RascalC.cov_utils import convert_cov_legendre
full_cov = convert_cov_legendre(results["full_theory_covariance"], max_l)